# Calculating SFH with Diffstar and DiffstarPop

This notebook gives two basic illustrations of how to use diffstar to model the SFHs of individual and populations of galaxies.

### SFH of an individual diffstar galaxy

In the cell below, we'll use the default diffmah and diffstar parameters, and then use the `sfh_singlegal` function to calculate the SFH.

In [ ]:
import numpy as np
from diffstar.defaults import DEFAULT_MAH_PARAMS
from diffstar.defaults import DEFAULT_DIFFSTAR_PARAMS

today_gyr = 13.8 
tarr = np.linspace(0.9, today_gyr, 100)

In [ ]:
from diffstar import calc_sfh_singlegal

sfh_gal = calc_sfh_singlegal(
    DEFAULT_DIFFSTAR_PARAMS, DEFAULT_MAH_PARAMS, tarr)

In [ ]:
from matplotlib import pyplot as plt

fig, ax = plt.subplots(1, 1)
ylim = ax.set_ylim(2e-3, 50)
yscale = ax.set_yscale('log')

__=ax.plot(tarr, sfh_gal, color='k')

xlabel = ax.set_xlabel(r'${\rm cosmic\ time\ [Gyr]}$')
ylabel = ax.set_ylabel(r'${\rm SFR\ [M_{\odot}/yr]}$')

### Generating populations of galaxy SFHs with DiffstarPop

DiffstarPop is a population-level model of Diffstar SFHs. DiffstarPop is formulated to capture $P(\theta_{\rm SFH}\vert\theta_{\rm MAH}),$
the PDF of diffstar properties, $\theta_{\rm SFH},$ conditioned on the diffmah parameters of halo mass assembly, $\theta_{\rm MAH}.$ Thus in the cells below, we will first use DiffmahPop to generate a cosmologically representative population of diffmah parameters for halos with Milky Way mass today, and then we will use DiffstarPop to create a Monte Carlo realization of the galaxy SFHs that live in the halos.

In [ ]:
from jax import random as jran
ran_key = jran.key(0)

In [ ]:
from diffmah.diffmahpop_kernels import mc_cenpop, DEFAULT_DIFFMAHPOP_PARAMS

n_halos = 500
lgm_obs = np.zeros(n_halos) + 12.0
t_obs = np.zeros(n_halos) + tarr[-1]
logt0 = np.log10(tarr[-1])

ran_key, mah_key = jran.split(ran_key, 2)
halopop = mc_cenpop(DEFAULT_DIFFMAHPOP_PARAMS, tarr, lgm_obs, t_obs, mah_key, logt0)

fig, ax = plt.subplots(1, 1)
yscale = ax.set_yscale('log')
for ih in range(5):
    __=ax.plot(tarr, 10**halopop.log_mah[ih, :])

xlabel = ax.set_xlabel(r'${\rm cosmic\ time\ [Gyr]}$')
ylabel = ax.set_ylabel(r'$M_{\rm halo}(t)\ {\rm [M_{\odot}]}$')

In [ ]:
from diffstar.diffstarpop import mc_diffstar_sfh_galpop, DEFAULT_DIFFSTARPOP_PARAMS

ZZ = np.zeros(n_halos)

ran_key, sfh_key = jran.split(ran_key, 2)
lgmu_infall = -1.0 + ZZ
logmhost_infall = 13.0 + ZZ
gyr_since_infall = 2.0 + ZZ
upids = np.zeros(n_halos).astype(int)-1

_res = mc_diffstar_sfh_galpop(
    DEFAULT_DIFFSTARPOP_PARAMS,
    halopop.mah_params,
    halopop.log_mah[:, -1],
    upids,
    lgmu_infall,
    logmhost_infall,
    gyr_since_infall,
    sfh_key,
    tarr,
)
sfh_q, sfh_ms = _res[2:4]
mc_is_q = _res[-1]

sfh = np.where(mc_is_q.reshape((n_halos, 1)), sfh_q, sfh_ms)

In [ ]:
fig, ax = plt.subplots(1, 1)
ylim = ax.set_ylim(2e-3, 50)
yscale = ax.set_yscale('log')

for igal in range(5):
    __=ax.plot(tarr, sfh[igal, :])


xlabel = ax.set_xlabel(r'${\rm cosmic\ time\ [Gyr]}$')
ylabel = ax.set_ylabel(r'${\rm SFR\ [M_{\odot}/yr]}$')